**After this assignment you will be able to:**
- Use non-linear units like ReLU to improve your model
- Build a deeper neural network (with more than 1 hidden layer)
- Implement an easy-to-use neural network class

# 1 - Packages

In [2]:
# packages
import numpy as np
import h5py
import matplotlib as plt
from testCases_v4a import *
from dnn_utils_v2 import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

# 2 - Outline of the Assignment

<img src="images/final outline.png" style="width:800px;height:500px;">

# 3 - Initialization

## 3.1 - 2-layter Neural Network
**Instructions**:
- The model's structure is: *LINEAR -> RELU -> LINEAR -> SIGMOID*. 
- Use random initialization for the weight matrices. Use `np.random.randn(shape)*0.01` with the correct shape.
- Use zero initialization for the biases. Use `np.zeros(shape)`.

In [3]:
def initialize_parameters(n_x, n_h, n_y):
    '''
    Arguments
    n_x : size of the input layer
    n_h : size of the hidden layer
    n_y : size of the output layer
    ========================================
    Returns
    W1 : weight matrix (n_h, n_x)
    b1 : bias vector (n_h, 1)
    W2 : weight matrix (n_y, n_h)
    b2 : bias vector (n_y, 1)
    '''
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {'W1':W1, 'b1':b1, 'W2':W2, 'b2':b2}
    
    return parameters    

## 3.2 - L-layer Neural Network


<table style='width:100%'>
    <tr>
        <td>  </td> 
        <td> Shape of W </td> 
        <td> Shape of b  </td> 
        <td> Activation </td>
        <td> Shape of Activation </td> 
    <tr>
    <tr>
        <td> Layer 1 </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> Layer 2 </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    <tr>
        <td> Layer L-1 </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>    
    <tr>
        <td> Layer L </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

In [4]:
def initialize_parameters_deep(layer_dims):
    '''
    Arguments
    layer_dims : array(list) of dimensions of each layer
    ======================================================
    Returns
    W1, b1, W2, b2, ...., WL, bL
    '''
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b'+str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters  

# 4 - Forward Propagation Module

## 4.1 - Linear forward

In [5]:
def linear_forward(A, W, b):
    '''
    Arguments
    A : activations from previous layer (or input when l=1)
    W : weight matrix (size of curr layer, size of prev layer)
    b : bias vector (size of curr layer, 1)
    ================================================================
    Returns
    Z : input of the activation function
    cache : tuple of (A, W, B) for backward propagation
    '''
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

## 4.2 - Linear - activation forward
- Sigmoid
- ReLU

In [6]:
def linear_activation_forward(A_prev, W, b, activation):
    '''
    Arguments
    A_prev : activations from previous layer (or input when l=1)
    W : weight matrix (size of curr layer, size of prev layer)
    b : bias vector (size of curr layer, 1)
    activation : 'sigmoid' or 'relu'
    ================================================================
    Returns:
    A : after activations
    cache : tuple of (A, W, B) for backward propagation
    '''
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

## 4.3 - L-layer Model

In [7]:
def L_model_forward(X, parameters):
    '''
    [LINEAR > RELU]*[L-1] > LINEAR > SIGMOID > OUTPUT
    ===================================================
    Arguments
    X : data (input size, number of examples)
    parameters : initialized parameters
    ===================================================
    Returns
    AL : last activated value
    caches
    '''
    caches = []
    A = X
    L = len(parameters) // 2 # there are 2 parameters for each layer
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev,
                                             parameters['W{}'.format(l)],
                                             parameters['b{}'.format(l)],
                                             activation='relu')
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], activation='sigmoid')
    caches.append(cache)
    
    return AL, caches

In [8]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


# 5 - Cost fnction
$${J} = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

In [9]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -(1/m) * np.sum(Y*np.log(AL) + (1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    
    return cost

In [10]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.2797765635793422


# 6 -  Backward Propagation Module

<img src="images/backprop_kiank.png" style="width:650px;height:250px;">

- LINEAR backward
- LINEAR -> ACTIVATION backward where ACTIVATION computes the derivative of either the ReLU or sigmoid activation
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (whole model)

## 6.1 - Linear backward
$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

In [11]:
def linear_backward(dZ, cache):
    '''
    Backward propagation for a single layer l
    ==============================================
    Arguments
    dZ : Gradients of the cost
    cache : tuple (A_prev, W, b)
    ==============================================
    Returns
    dA_prev : Gradient of the cost with respect to the activation
    dW      : Gradient of the cost with respect to W
    db      : Gradient of the cost with respect to b
    '''
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

## 6.2 - Linear-activation backward
- Sigmoid
- ReLU

In [12]:
def linear_activation_backward(dA, cache, activation):
    '''
    Arguments
    dA : Post-activation gradient for current layer l
    cache : (linear_cache, activation_cache)
    activation : 'sigmoid' or 'relu'
    ====================================================
    Returns
    dA_prev
    dW
    db
    '''
    linear_cache, activation_cache = cache
    
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

## 6.3 L-moel backward
<img src="images/mn_backward.png" style="width:450px;height:300px;">

In [14]:
def L_model_backward(AL, Y, caches):
    """
    [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID
    ============================================================
    Arguments:
    AL : probability vector (output)
    Y  : true "label" vector
    caches
    ============================================================
    Returns:
    grads 
    """
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, 'relu')
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

## 6.4 - Update parameters

In [16]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2

    for l in range(L):
        parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - learning_rate * grads["dW"+str(l+1)]
        parameters["b"+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * grads["db"+str(l+1)]

    return parameters